In [1]:
# getting data through DataLoaders
import numpy as np
import torch
from loadImgsAndLabels import loadloadImgsAndLabels
from torch.utils.data import TensorDataset, DataLoader

def prepare_mnist_loaders(x_train, y_train, x_test, y_test, batch_size, test_batch_size, val_split):
	x_train, x_val = np.split(x_train, [val_split])
	y_train, y_val = np.split(y_train, [val_split])

	x_train, y_train, x_val, y_val, x_test, y_test = map(
		torch.tensor,
		(x_train, y_train, x_val, y_val, x_test, y_test)
	)
	train_ds = TensorDataset(x_train, y_train)
	val_ds = TensorDataset(x_val, y_val)
	test_ds = TensorDataset(x_test, y_test)

	train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
	val_dl = DataLoader(val_ds, batch_size=test_batch_size)
	test_dl = DataLoader(test_ds, batch_size=test_batch_size)

	return train_dl, val_dl, test_dl


batch_size = 64
test_batch_size = 2 * batch_size
val_split = 50000

x_train, y_train = loadloadImgsAndLabels(
	'../datasets/train-images.idx3-ubyte',
	'../datasets/train-labels.idx1-ubyte'
)

x_test, y_test = loadloadImgsAndLabels(
	'../datasets/t10k-images.idx3-ubyte',
	'../datasets/t10k-labels.idx1-ubyte'
)

train_dl, val_dl, test_dl = prepare_mnist_loaders(
	x_train, y_train, x_test, y_test, batch_size, test_batch_size, val_split
)

In [ ]:
from torch import optim
from torch import nn

lr = 0.5

class Mnist_Logistic(nn.Module):
	def __init__(self):
		super().__init__()
		self.lin = nn.Linear(784, 10)

	def forward(self, xb):
		return self.lin(xb)
    
model = Mnist_Logistic()
opt = optim.SGD(model.parameters(), lr=lr)

In [ ]:


def loss_batch(model, loss_func, xb, yb, opt=None):
	loss = loss_func(model(xb), yb)

	if opt is not None:
		loss.backward()
		opt.step()
		opt.zero_grad()

	return loss.item(), len(xb)

def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
	for epoch in range(epochs):
		model.train()
		for xb, yb in train_dl:
			loss_batch(model, loss_func, xb, yb, opt)

		model.eval()
		with torch.no_grad():
			losses, nums = zip(
				*[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
			)
		val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

		print(epoch, val_loss)


model = Mnist_Logistic()
opt = optim.SGD(model.parameters(), lr=lr)

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")